In [ ]:
import pandas as pd
from models.titulo import Titulo


df = pd.read_csv(r'C:\Users\guilherme.seehagen\OneDrive - UNIOESTE\Área de Trabalho\mq3_trabalho\data\tesouro_direto_titulos.csv')


df['Rentabilidade anual'] = df['Rentabilidade anual'].astype(float) / 100

# Cria os títulos
titulos = [
    Titulo(
        nome=row['Título'],
        vencimento=row['Vencimento'],
        taxa_juros_anual=row['Rentabilidade anual']
    )
    for _, row in df.iterrows()
]

# Teste
titulos[2].resumo_do_titulo()

A taxa anual é 0.141
O vencimento do título é 01/01/2035
O nome do título é Tesouro Prefixado com juros semestrais 2035
Sua taxa mensal equivalente é 0.011052724217629706
